# Lab 1 Part I: MNIST 2D CNN

## Imports

In [30]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Load and format MNIST dataset

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test  = x_test.astype('float32')  / 255
x_train = x_train.reshape((60000, 28, 28, 1)) # 'channels_last' format
x_test  = x_test.reshape((10000, 28, 28, 1)) # 'channels_last' format
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test,  10)

## Build model

In [33]:
lay1=Conv2D(2,kernel_size=(1,1),activation='relu',padding='same')(x_train)
lay1=Conv2D(2,kernel_size=(7,7),strides=(2,2),activation='relu',padding='same')(lay1)
lay1=Conv2D(2,kernel_size=(1,1),activation='relu',padding='same')(lay1)
lay1=MaxPooling2D(pool_size=(7,7),strides=(2,2),padding='same')(lay1)
lay2=Conv2D(4,kernel_size=(1,1),activation='relu',padding='same')(x_train)
lay2=Conv2D(4,kernel_size=(7,7),strides=(2,2),activation='relu',padding='same')(lay2)
lay2=Conv2D(4,kernel_size=(1,1),activation='relu',padding='same')(lay2)
lay2=MaxPooling2D(pool_size=(7,7),strides=(2,2),padding='same')(lay2)
lay3=Conv2D(6,kernel_size=(1,1),activation='relu',padding='same')(x_train)
lay3=Conv2D(6,kernel_size=(7,7),strides=(2,2),activation='relu',padding='same')(lay3)
lay3=Conv2D(6,kernel_size=(1,1),activation='relu',padding='same')(lay3)
lay3=MaxPooling2D(pool_size=(7,7),strides=(2,2),padding='same')(lay3)
fc=concatenate([lay1,lay2,lay3])
fc=Flatten()(fc)
fc=Dense(10,activation='relu')(fc)
outputs=Dense(10,activation='softmax')(fc)
model=Sequential(input=x_train,output=outputs)
model.compile(loss=keras.losses.categorical_crossentropy,
  optimizer=keras.optimizers.Adadelta(),
  metrics=['accuracy'])
model.summary()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2778/3474325606.py", line 17, in <module>
    model=Sequential(input=x_train,output=outputs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/trackable/base.py", line 204, in _method_wrapper
    result = method(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 67, in error_handler
    filtered_tb = _process_traceback_frames(e.__traceback__)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Sequential.__init__() got an une

## Train model

In [22]:
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', min_delta=0.003)
mc = ModelCheckpoint('best_model.h5', monitor='val_categorical_accuracy', mode='max', save_best_only=True)
model.fit(x_train, y_train, epochs=25, validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/25
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1476 - categorical_accuracy: 0.9568 - val_loss: 0.1419 - val_categorical_accuracy: 0.9574
Epoch 2/25
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1417 - categorical_accuracy: 0.9584 - val_loss: 0.1498 - val_categorical_accuracy: 0.9541


## Evaluate model

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

## Save trained model

In [ ]:
model.save('lab1_part1_mnist.h5')